In [ ]:
!pip install ipywidgets
!pip install --upgrade jupyter_client
!pip install transformers
!pip install datasets
!pip install librosa
!pip install torchaudio
!pip install jiwer
!pip install wandb
!git config --global credential.helper store
!apt-get install git-lfs 

import librosa
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer, Wav2Vec2Model, Wav2Vec2Config
import numpy as np
import os
import random
from ipywidgets import *
from IPython.display import display, HTML
from datasets import load_dataset, load_metric, Dataset, ClassLabel
import datasets 
import re
import json
import IPython.display as ipd
import pandas as pd
import wandb

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
wandb.login()

In [ ]:
#support functions

#special character removal
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\(\)\«\»\’\/\'\%\x08\&\_\–\d{10}\ê\é\à\ß\â\ç\è"]'
def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

#extract characters for vocab dict
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

#prepare audio dataset
def prepare_dataset(batch):
    audio = batch['audio_path']

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    #batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

#create data collator CTC
#code from https://huggingface.co/blog/fine-tune-wav2vec2-english by Patrick von Platen
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch
    
#define metric Word Error Rate (WER)  
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    
def map_to_result(batch):
      with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

      pred_ids = torch.argmax(logits, dim=-1)
      batch["pred_str"] = processor.batch_decode(pred_ids)[0]
      batch["text"] = processor.decode(batch["labels"], group_tokens=False)

      return batch

In [ ]:
random.seed(42)
np.random.seed(42)
#already run = "toy_train_data.csv",
            #"toy_train_data_augment_0.1.csv",
           # "toy_train_data_augmented.csv","toy_train_data_fast_10pct.csv","toy_train_data_masked_audio_10ms.csv","toy_train_data_masked_audio.csv",
            #"toy_train_data_slow_10pct.csv","toy_train_data_random_low_pass.csv","toy_train_data_random_noise_0.1.csv", "toy_train_data_random_high_pass.csv","toy_train_data_random_noise.csv",
csv_name = "full_train.csv"
            

#model_list = ["facebook/wav2vec2-base","facebook/wav2vec2-large-xlsr-53","Yves/wav2vec2-large-xlsr-53-swiss-german"]
model_name  = "facebook/wav2vec2-large-xlsr-53"
    


wandb.init(project="Last_Run",name=model_name+"_"+csv_name[:-4], entity="scasutt")

#load dataset used for training  
feature_dict = {"audio_path": datasets.Audio(sampling_rate=16_000),"text": datasets.Value("string")}
data_features = datasets.Features(feature_dict)

dataset = load_dataset("csv", 
                        data_files={"train": csv_name,
                                    "test":"test_augment_final.csv"}  

                        )
#set audio column and delete unused data
dataset = dataset.cast_column("audio_path", datasets.Audio(sampling_rate=16_000))
dataset = dataset.remove_columns(["Unnamed: 0"])
#remove special characters
dataset = dataset.map(remove_special_characters)
#extract characters 
vocabs = dataset.map(extract_all_chars, 
                    batched=True, 
                    batch_size=-1, 
                    keep_in_memory=True, 
                    remove_columns=dataset.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

#dump vocab to json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

#prepare dataset for training
dataset = dataset.map(prepare_dataset, remove_columns = dataset.column_names["train"], num_proc=4)

#define data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


#set up model
model = Wav2Vec2ForCTC.from_pretrained(
                                    model_name,
                                    attention_dropout=0.1,
                                    hidden_dropout=0.1,
                                    feat_proj_dropout=0.0,
                                    mask_time_prob=0.05,
                                    layerdrop=0.1,
                                    pad_token_id=processor.tokenizer.pad_token_id,
                                    vocab_size=len(processor.tokenizer),
                                    ctc_loss_reduction="mean"

                                    )

model.config.ctc_zero_infinity = True
#name for model save on hub

output_dir = model_name+"_"+csv_name[:-4]

#set up training arguments 20 epoch training
training_args = TrainingArguments(
                                    output_dir=output_dir,
                                    push_to_hub=True,
                                    group_by_length=True,
                                    per_device_train_batch_size=16,
                                    gradient_accumulation_steps=16,
                                    evaluation_strategy="steps",
                                    num_train_epochs=10,
                                    fp16=True,
                                    gradient_checkpointing=True,
                                    save_steps=500,
                                    eval_steps=1000,
                                    logging_steps=500,
                                    learning_rate=1e-4,
                                    weight_decay=0.005,
                                    warmup_steps=1000,
                                    save_total_limit=1,

                                    )


#set up trainer
trainer = Trainer(
                    model=model,
                    data_collator=data_collator,
                    args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=dataset["train"],
                    eval_dataset=dataset["test"],
                    tokenizer=processor.feature_extractor,

                    )
#empty cuda cache
torch.cuda.empty_cache()

#train model
trainer.train()



#save results
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

results_data = pd.DataFrame.from_dict(results)
results_data.to_csv(model_name[9:]+"_"+csv_name[:-4]+"_results.csv")

trainer.push_to_hub()

#save model to hub
tokenizer.push_to_hub("scasutt/"+model_name[9:]+"_"+csv_name[:-4])
feature_extractor.push_to_hub("scasutt/"+model_name[9:]+"_"+csv_name[:-4])
processor.push_to_hub("scasutt/"+model_name[9:]+"_"+csv_name[:-4])
         
    